In [3]:
from load_data import load_expert_data
from load_data import load_experiment_data
from utils_analysis import load_analysis
from utils_analysis import sort_by_key
#from aggregate_labels import get_top_labels
from analyze_workers import get_worker_analysis
from calculate_iaa import get_agreement, get_kappa_pairs
from calculate_iaa import create_matrix
from calculate_iaa import get_full_report

import csv

In [6]:

run = 4
group = 'expert_inspection1'
n_q = '*'
batch = '*'
data_dict_list = load_expert_data(run, group, n_q, batch)


name = f'run{run}-group_{group}-batch{batch}'.replace('*', '-all-')
df, filepath = get_worker_analysis(data_dict_list, name)
df.drop(['annotations'], axis = 1, inplace=True)
df[['workerid', 'contradiction_poss_contradiction_ratio', 'n_contradictions', 'n_fails', 'n_annotations']]

Discarded 0.0 annotations.


,workerid,contradiction_poss_contradiction_ratio,n_contradictions,n_fails,n_annotations
0,antske_run1b,0.0,0,0,30
1,pia run 1,0.0,0,0,30
2,Piek run-1,0.0,0,0,30
3,antske_run1,0.0,0,0,5


## Agreement overview

In [7]:
# Agreement collapsed

full_report = get_full_report(data_dict_list)
for collapsed_set, scores in full_report.items():
    print(collapsed_set)
    for n, s in scores.items():
        print(n, s)
    print()

full
Krippendorff 0.6777142857142857
Proportional 0.8722222222222221
Av_Cohens_kappa 0.6258697087731723

pos_neg
Krippendorff 1.0
Proportional 1.0
Av_Cohens_kappa 1.0

levels
Krippendorff 1.0
Proportional 0.8
Av_Cohens_kappa 1.0

similar_relations
Krippendorff 0.7082228116710876
Proportional 0.6666666666666666
Av_Cohens_kappa 0.7909090909090909



## Agreement by annotator pair

In [8]:

matrix = create_matrix(data_dict_list)
pair_kappa_dict = get_kappa_pairs(matrix)

for pair, k in pair_kappa_dict.items():
    if 'antske_run1' not in pair:
        print(pair, k)

('Piek run-1', 'antske_run1b') 0.8305084745762712
('Piek run-1', 'pia run 1') 0.6610169491525424
('antske_run1b', 'pia run 1') 0.6273291925465838


# Analyzing disagreements

In [9]:
report_rows = []
data_by_pair  = sort_by_key(data_dict_list, ['property', 'concept'])
for pair, dicts in data_by_pair.items():

    relation_dicts = sort_by_key(dicts, ['relation'])
    for rel, dicts in relation_dicts.items():
        answers = set(d['answer'] for d in dicts if d['workerid'])
        if len(answers) != 1:
            for d in dicts:
                row_d = dict()
                row_d['triple'] = rel+'-'+pair
                row_d['worker'] = d['workerid']
                row_d['answer'] = d['answer']
                #print(pair, rel, '\t', d['workerid'], d['answer'])
                report_rows.append(row_d)
        else:
            print(rel+'-'+pair)
            #print()

typical_of_property-fly-arrow
typical_of_concept-fly-arrow
afforded_usual-fly-arrow
variability_limited-fly-arrow
creative-fly-arrow
rare-fly-arrow
unusual-fly-arrow
impossible-fly-arrow
creative-roll-pin
unusual-roll-pin
impossible-roll-pin
implied_category-yellow-buttercup
typical_of_property-yellow-buttercup
typical_of_concept-yellow-buttercup
variability_limited-yellow-buttercup
variability_open-yellow-buttercup
typical_of_concept-roll-tire
afforded_unusual-roll-tire
afforded_usual-roll-tire
variability_limited-roll-tire
creative-roll-tire
rare-roll-tire
unusual-roll-tire
impossible-roll-tire


In [10]:
header = report_rows[0].keys()
with open(f'../analyses/expert_annotations/report_{group}.csv', 'w') as outfile:
    writer = csv.DictWriter(outfile, fieldnames = header)
    writer.writeheader()
    for r in report_rows:
        writer.writerow(r)